# Liabraries

In [4]:
import imaplib
import email
import re
import datetime
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build

- Set up the connection to the Email account

In [5]:
# Set up the connection to the email account
imap_server = "imap.gmail.com"
imap_user = "testingforcode71@gmail.com"
imap_pass = "nxol npsi xhux uvxq"
imap = imaplib.IMAP4_SSL(imap_server)
imap.login(imap_user, imap_pass)

('OK', [b'testingforcode71@gmail.com authenticated (Success)'])

In [8]:
# Select the mailbox where the notifications are stored
mailbox = "INBOX"
imap.select(mailbox)

('OK', [b'6'])

In [38]:
# Search for emails matching certain criteria
search_criteria = "(FROM 'hafizsohaib478@gmail.com' SUBJECT 'Sohaib')"
result, data = imap.search(None, search_criteria)

In [46]:
# Extract data from the emails and store it in a dictionary
data_dict = {}
for email_id in data[0].split():
    _, email_data = imap.fetch(email_id, "(RFC822)")
    _, email_body = email_data[0]
    email_message = email.message_from_bytes(email_body)

    # Extract relevant data from email message
    timestamp = datetime.datetime.strptime(email_message['Date'], '%a, %d %b %Y %H:%M:%S %z').strftime('%Y-%m-%d %H:%M:%S')
    subject = email_message['Subject']
    body = email_message.get_payload()
    # Use regex to extract data from the email body
    match = re.search(r'Your data is: (\d+)', body)
    data_point = match.group(1)

    # Add data to dictionary
    data_dict[timestamp] = {'Subject': subject, 'Data': data_point}

In [48]:
# Connect to Google Sheets API
scopes = ['https://www.googleapis.com/auth/spreadsheets']
creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scopes)
sheets_service = build('sheets', 'v4', credentials=creds)
drive_service = build('drive', 'v3', credentials=creds)

In [ ]:
# Access the desired spreadsheet and worksheet
spreadsheet_name = 'My Spreadsheet'
worksheet_name = 'My Worksheet'
sheet_metadata = sheets_service.spreadsheets().get(spreadsheetId='SPREADSHEET_ID').execute()
sheet_id = sheet_metadata['sheets'][0]['properties']['sheetId']
range_name = f'{worksheet_name}!A1:C{len(data_dict) + 1}'

In [ ]:
# Write data to Google Sheets
values = []
for timestamp, data in data_dict.items():
    values.append([timestamp, data['Subject'], data['Data']])

body = {'values': values}
result = sheets_service.spreadsheets().values().update(
    spreadsheetId='SPREADSHEET_ID',
    range=range_name,
    valueInputOption='USER_ENTERED',
    body=body).execute()

print(f"{result.get('updatedCells')} cells updated.")

# Close the connection to the email account
imap.close()
imap.logout()